In [1]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar xf aclImdb_v1.tar.gz

--2023-04-24 09:41:35--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  15.6MB/s    in 8.1s    

2023-04-24 09:41:43 (9.88 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [2]:
import glob
import pathlib
import re
import torch

remove_marks_regex = re.compile("[,\.\(\)\[\]\*:;]|<.*?>")
shift_marks_regex = re.compile("([?!])")

In [3]:
def text2ids(text, vocab_dict):  # 긴 문자열을 토큰 ID 리스트로 변환하는 함수
  # !? 이외의 기호 삭제
  text = remove_marks_regex.sub("",text)
  # !?와 단어사이에 공백 삽입
  text = shift_marks_regex.sub(r" \1 ",text)
  tokens = text.split()
  return [ vocab_dict.get(token,0) for token in tokens]  # 용어집에 포함되지 않는 토큰은 0으로 채운다


In [4]:
#ID리스트를 int64의 텐서로 변환하는 함수 변환할때는 각 문장을 분할한 후 토근 수를 제한하고, 반대로 그 수에 
# 미치지 못하면 뒤를 0d로 채운다
def list2tensor(token_idxes, max_len=100, padding=True): 
  if len(token_idxes) > max_len:
    token_idxes = token_idxes[:max_len]
  n_tokens = len(token_idxes)
  if padding:
    token_idxes = token_idxes + [0]*(max_len - len(token_idxes))
  return torch.tensor(token_idxes, dtype = torch.int64), n_tokens

In [5]:
# Dataset 클래스 작성

In [6]:
from torch import nn, optim
from torch.utils.data import Dataset,DataLoader,TensorDataset
import tqdm

In [7]:
class IMDBDataset(Dataset):
    def __init__(self, dir_path, train=True,
                 max_len=100, padding=True):
        self.max_len = max_len
        self.padding = padding
        
        path = pathlib.Path(dir_path)
        vocab_path = path.joinpath("imdb.vocab")
        
        # 용어집 파일을 읽어서 행 단위로 분할
        self.vocab_array = vocab_path.open() \
                            .read().strip().splitlines()
        # 단어가 키이고 값이 ID인 dict 만들기
        self.vocab_dict = dict((w, i+1) \
            for (i, w) in enumerate(self.vocab_array))
        
        if train:
            target_path = path.joinpath("train")
        else:
            target_path = path.joinpath("test")
        pos_files = sorted(glob.glob(
            str(target_path.joinpath("pos/*.txt"))))
        neg_files = sorted(glob.glob(
            str(target_path.joinpath("neg/*.txt"))))
        # pos는 1, neg는 0인 label을 붙여서
        # (file_path, label)의 튜플 리스트 작성
        self.labeled_files = \
            list(zip([0]*len(neg_files), neg_files )) + \
            list(zip([1]*len(pos_files), pos_files))
    
    @property
    def vocab_size(self):
        return len(self.vocab_array)
    
    def __len__(self):
        return len(self.labeled_files)
    
    def __getitem__(self, idx):
        label, f = self.labeled_files[idx]
        # 파일의 텍스트 데이터를 읽어서 소문자로 변환
        data = open(f).read().lower()
        # 텍스트 데이터를 ID 리스트로 변환
        data = text2ids(data, self.vocab_dict)
        # ID 리스트를 Tensor로 변환
        data, n_tokens = list2tensor(data, self.max_len, self.padding)
        return data, label, n_tokens

In [8]:
# 훈련용과 테스트용 DataLoader 작성
train_data = IMDBDataset('/content/aclImdb')
test_data = IMDBDataset('/content/aclImdb',train=False)
train_loader = DataLoader(train_data,batch_size=32,shuffle=True)
test_loader = DataLoader(test_data,batch_size=32,shuffle=False)

In [9]:
data = next(iter(train_loader))
data

[tensor([[    9,   336,     5,  ...,    63,   699,    10],
         [    9,    80,  2359,  ...,   136,  1455,   382],
         [  198,   205,   489,  ...,     0,     0,     0],
         ...,
         [    0,     4,     1,  ...,     1,   116,   405],
         [   10,     6,     3,  ...,     5,    64, 15036],
         [   10,    16,  1306,  ...,  1533,     2,  2196]]),
 tensor([1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0,
         0, 1, 0, 0, 0, 1, 1, 0]),
 tensor([100, 100,  62, 100, 100, 100, 100, 100,  81, 100, 100, 100, 100, 100,
         100, 100, 100, 100,  90, 100, 100, 100, 100, 100, 100, 100, 100, 100,
         100, 100, 100, 100])]

In [10]:
# 신경망..  x가 입력되었을때 0또는 1일 출력되는 2진 분류 문제
# x를 Embedding 으로 벡터 시계열로 변환하고, RNN에 넣어서 마지막 출력을 1차원 선형 계층과 연결

In [11]:
class SequenceTaggingNet(nn.Module):
    def __init__(self, num_embeddings,
                 embedding_dim=50, 
                 hidden_size=50,
                 num_layers=1,
                 dropout=0.2):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_dim,
                            padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim,
                            hidden_size, num_layers,
                            batch_first=True, dropout=dropout)
        self.linear = nn.Linear(hidden_size, 1)


        
        
    def forward(self, x, h0=None, l=None):
        # ID를 Embedding으로 다차원 벡터로 변환
        # x는 (batch_size, step_size) 
        # -> (batch_size, step_size, embedding_dim)
        x = self.emb(x)
        # 초기 상태 h0와 함께 RNN에 x를 전달
        # x는(batch_size, step_size, embedding_dim)
        # -> (batch_size, step_size, hidden_dim)
        x, h = self.lstm(x, h0)
        # 마지막 단계만 추출
        # xは(batch_size, step_size, hidden_dim)
        # -> (batch_size, 1)
        if l is not None:
            # 입력의 원래 길이가 있으면 그것을 이용
            x = x[list(range(len(x))), l-1, :]
        else:
            # 없으면 단순히 마지막 것을 이용
            x = x[:, -1, :]
        # 추출한 마지막 단계를 선형 계층에 넣는다
        x = self.linear(x)
        # 불필요한 차원을 삭제
        # (batch_size, 1) -> (batch_size, )
        x = x.squeeze()
        return x

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# 훈련 및 평가

def eval_net(net, data_loader, device="cpu"):
    net.eval()
    ys = []
    ypreds = []
    for x, y, l in data_loader:
        x = x.to(device)
        y = y.to(device)
        l = l.to(device)
        with torch.no_grad():
            y_pred = net(x, l=l)
            y_pred = (y_pred > 0).long()
            ys.append(y)
            ypreds.append(y_pred)
    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)
    acc = (ys == ypreds).float().sum() / len(ys)
    return acc.item()

In [ ]:

from statistics import mean

# num_embeddings에는 0을 포함해서 train_data.vocab_size+1를 넣는다
net = SequenceTaggingNet(train_data.vocab_size+1, 
num_layers=2)
net.to(device)
opt = optim.Adam(net.parameters())
loss_f = nn.BCEWithLogitsLoss()

for epoch in range(10):
    losses = []
    net.train()
    for x, y, l in tqdm.tqdm(train_loader):
        x = x.to(device)
        y = y.to(device)
        l = l.to(device)
        y_pred = net(x, l=l)
        loss = loss_f(y_pred, y.float())
        net.zero_grad()
        loss.backward()
        opt.step()
        losses.append(loss.item())
    train_acc = eval_net(net, train_loader, device)
    val_acc = eval_net(net, test_loader, device)
    print(epoch, mean(losses), train_acc, val_acc)

100%|██████████| 782/782 [00:12<00:00, 64.14it/s]


0 0.6761752057563314 0.5744400024414062 0.5739200115203857


100%|██████████| 782/782 [00:09<00:00, 84.08it/s]


1 0.6872068557440473 0.5701999664306641 0.5543599724769592


100%|██████████| 782/782 [00:09<00:00, 85.45it/s]


...

100%|██████████| 782/782 [00:09<00:00, 85.91it/s]


4 0.32744543978472807 0.9059999585151672 0.7905600070953369


100%|██████████| 782/782 [00:08<00:00, 90.62it/s]


5 0.25656775746237287 0.9384399652481079 0.7973200082778931


100%|██████████| 782/782 [00:09<00:00, 81.24it/s]
